In [1]:
# Libraries to be used are included
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import *
import copy
import math
%matplotlib inline

In [2]:
# Data uploaded
data = pd.read_csv("health care diabetes.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
# Split the data values and dependent variable
x_train = data.drop("Outcome", axis=1).values
y_train = data["Outcome"].values

In [5]:
print("Type of x_train:", type(x_train))
print("Type of y_train:", type(y_train))

Type of x_train: <class 'numpy.ndarray'>
Type of y_train: <class 'numpy.ndarray'>


In [6]:
print("The shape of x_train is:", x_train.shape)
print("The shape of y_train is:", y_train.shape)

The shape of x_train is: (768, 8)
The shape of y_train is: (768,)


In [7]:
print("First five elements of x_train:", x_train[:5])
print("First five elements of y_train:", y_train[:5])
print("Number of training examples(m):", len(x_train))

First five elements of x_train: [[6.000e+00 1.480e+02 7.200e+01 3.500e+01 0.000e+00 3.360e+01 6.270e-01
  5.000e+01]
 [1.000e+00 8.500e+01 6.600e+01 2.900e+01 0.000e+00 2.660e+01 3.510e-01
  3.100e+01]
 [8.000e+00 1.830e+02 6.400e+01 0.000e+00 0.000e+00 2.330e+01 6.720e-01
  3.200e+01]
 [1.000e+00 8.900e+01 6.600e+01 2.300e+01 9.400e+01 2.810e+01 1.670e-01
  2.100e+01]
 [0.000e+00 1.370e+02 4.000e+01 3.500e+01 1.680e+02 4.310e+01 2.288e+00
  3.300e+01]]
First five elements of y_train: [1 0 1 0 1]
Number of training examples(m): 768


### Sigmoid function

Recall that for logistic regression, the model is represented as

$$ f_{\mathbf{w},b}(x) = g(\mathbf{w}\cdot \mathbf{x} + b)$$
where function $g$ is the sigmoid function. The sigmoid function is defined as:

$$g(z) = \frac{1}{1+e^{-z}}$$


In [8]:
def sigmoid(z):
    g = 1 / (1 + math.e ** -z)
    return g

In [9]:
# Test for sigmoid function
print ("sigmoid([ -1, 0, 1, 2]) = " + str(sigmoid(np.array([-1, 0, 1, 2]))))

sigmoid([ -1, 0, 1, 2]) = [0.26894142 0.5        0.73105858 0.88079708]


## Cost Function

The cost function is calculated with following formulas

$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$


$$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$

In [10]:
def compute_cost(x, y, w, b, *argv):
    m, n = x.shape # m is row n is column
    
    total_cost = 0
    f_wb = 0
    
    for i in range(m):
        f_wb = 0
        for j in range(n):
            f_wb_ij = w[j] * x[i][j]
            f_wb += f_wb_ij
        f_wb += b
        loss = sigmoid(f_wb)
        total_cost += (-y[i] * math.log(loss)) - ((1 - y[i]) * math.log(1 - loss))
        
    total_cost /= m
    return total_cost

## Gradient Descent

The gradient descent is calculated with following formulas

$$
\frac{\partial J(\mathbf{w},b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)}) \tag{2}
$$
$$
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)})x_{j}^{(i)} \tag{3}
$$

In [11]:
def compute_gradient(x, y, w, b, *argv):
    m, n = x.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0
    
    for i in range(m):
        z_wb = 0
        for j in range(n):
            z_wb += w[j] * x[i][j]
        z_wb += b
        f_wb = sigmoid(z_wb)
        
        dj_db_i = f_wb - y[i]
        dj_db += dj_db_i
        
        for j in range(n):
            dj_dw_ij = (f_wb - y[i]) * x[i][j]
            dj_dw[j] += dj_dw_ij
            
    dj_dw /= m
    dj_db /= m

        
    return dj_db, dj_dw

In [12]:
def gradient_descent(x, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    
    m = len(x)
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(x, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(x, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

In [13]:
np.random.seed(1)
initial_w = 0.01 * (np.random.rand(8) - 0.5)
initial_b = -8

# Some gradient descent settings
iterations = 10000
alpha = 0.0001

w,b, J_history,_ = gradient_descent(x_train ,y_train, initial_w, initial_b, 
                                   compute_cost, compute_gradient, alpha, iterations, 0)

Iteration    0: Cost     2.55   
Iteration 1000: Cost     0.49   
Iteration 2000: Cost     0.48   
Iteration 3000: Cost     0.48   
Iteration 4000: Cost     0.48   
Iteration 5000: Cost     0.48   
Iteration 6000: Cost     0.48   
Iteration 7000: Cost     0.48   
Iteration 8000: Cost     0.48   
Iteration 9000: Cost     0.48   
Iteration 9999: Cost     0.48   


In [14]:
def predict(x, w, b): 

    # number of training examples
    m, n = x.shape   
    p = np.zeros(m)
   
    for i in range(m):   
        z_wb = 0
        for j in range(n): 
            z_wb += w[j] * x[i][j]
    
        z_wb += b
        
        # Calculate the prediction
        f_wb = sigmoid(z_wb)

        p[i] = 0 if f_wb < 0.5 else 1
        
    return p

In [15]:
p = predict(x_train, w,b)
print('Train Accuracy: %f'%(np.mean(p == y_train) * 100))

Train Accuracy: 77.213542
